In [1]:
from collections import defaultdict, Counter
import json
import pandas as pd
import time
from datetime import datetime

## Behance

In [10]:
# Bring the dataset
data = pd.read_csv("raw_dataset/Behance_appreciate_1M", sep=" ", header=None, names=['userId', 'itemId', 'time'])

In [13]:
# Count the number of users and items
user_count = defaultdict(lambda: 0)
item_count = defaultdict(lambda: 0)
for k, v in data.iterrows():
    user_count[v['userId']] += 1
    item_count[v['itemId']] += 1

In [17]:
# Remove the # user < 5 and # item < 5
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0
User = dict()
for k,v in data.iterrows():
    session_id = v['userId']; item = v['itemId']; time_ = v['time']
    if user_count[session_id] < 5 or item_count[item] < 5:
        continue
    
    if session_id in usermap:
        userid = usermap[session_id]
    else:
        usernum += 1
        userid = usernum
        usermap[session_id] = userid
        User[userid] = []
    if item in itemmap:
        itemid = itemmap[item]
    else:
        itemnum += 1
        itemid = itemnum
        itemmap[item] = itemid
    User[userid].append([time_, itemid])
    
# Sort the sequences
for userid in User.keys():
    User[userid].sort(key=lambda d:d[0])

In [18]:
# Save
f = open(f"dataset/Behance.txt", 'w')
for user in User.keys():
    for i in User[user]:
        f.write('%d %d %d\n' % (user, i[1], i[0]))
f.close()

## FSQ

In [2]:
# Bring the dataset and Count the number of users and items
rating_info = defaultdict(list)
user_count = defaultdict(int)
item_count = defaultdict(int)
with open("raw_dataset/fsq_checkins.dat", "r") as f:
    data = f.readlines()
    for i in data:
        i = i.split('|')
        user_id = int(i[1].strip())
        user_count[user_id] += 1
        item_id = int(i[2].strip())
        item_count[item_id] += 1

In [3]:
# Remove the interactions of # user < 5 and # item < 5
user_map = {}
user_num = 0
item_map = {}
item_num = 0
with open("fsq_checkins.dat", "r") as f:
    data = f.readlines()
    for i in data:
        i = i.split('|')
        user_id = int(i[1].strip())
        item_id = int(i[2].strip())
        if user_count[user_id] < 5 or item_count[item_id] < 5:
            continue
        if user_id not in user_map.keys():
            user_num += 1
            user_map[user_id] = user_num
        if item_id not in item_map.keys():
            item_num += 1
            item_map[item_id] = item_num 
        s_time = i[5]
        year = int(s_time[:5])
        month = int(s_time[6:8])
        day = int(s_time[9:11])
        hour = int(s_time[12:14])
        minute = int(s_time[15:17])
        timestamp = time.mktime(datetime.strptime(f"{year}/{month}/{day}/{hour}/{minute}", "%Y/%m/%d/%H/%M").timetuple())
        rating_info[user_map[user_id]].append((item_map[item_id], timestamp))

In [4]:
for k,v in rating_info.items():
    sort_rating = sorted(v, key=lambda b:b[1])
    rating_info[k] = sort_rating
    
f = open(f"dataset/fsq.txt", 'w')
for user in rating_info.keys():
    for i in rating_info[user]:
        f.write('%d %d %d\n' % (user, i[0], i[1]))
f.close()

## Amazon

In [3]:
import gzip
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield json.loads(l)

In [10]:
dataset_name = "reviews_Digital_Music.json.gz"
dataset_save_name = "Music"

In [12]:
countU = defaultdict(lambda: 0)
countP = defaultdict(lambda: 0)
line = 0
for l in parse(f"raw_dataset/{dataset_name}"):
    line += 1
    asin = l['asin']
    rev = l['reviewerID']
    time = l['unixReviewTime']
    countU[rev] += 1
    countP[asin] += 1

usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0
User = dict()
 
for l in parse(f"raw_dataset/{dataset_name}"):
    line += 1
    asin = l['asin']
    rev = l['reviewerID']
    time = l['unixReviewTime']
    if countU[rev] < 5 or countP[asin] < 5:
        continue

    if rev in usermap:
        userid = usermap[rev]
    else:
        usernum += 1
        userid = usernum
        usermap[rev] = userid
        User[userid] = []
    if asin in itemmap:
        itemid = itemmap[asin]
    else:
        itemnum += 1
        itemid = itemnum
        itemmap[asin] = itemid
    User[userid].append([time, itemid])

In [13]:
# Sort the sqeuences
for userid in User.keys():
    User[userid].sort(key=lambda x: x[0])
    
f = open(f"dataset/{dataset_save_name}.txt", 'w')
for user in User.keys():
    for i in User[user]:
        f.write('%d %d %d\n' % (user, i[1], i[0]))
f.close()